In [ ]:
import cv2
import os

def extract_frames_from_video(video_path, output_dir, frame_rate=1):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame_filename = os.path.join(output_dir, f"frame_{count}.jpg")
            cv2.imwrite(frame_filename, frame)
        count += 1
    cap.release()

# Example usage
if __name__ == "__main__":
    video_path = "data/video/crime_scene_video.mp4"
    output_dir = "data/frames"
    extract_frames_from_video(video_path, output_dir)


In [ ]:
import torch
from deep_sort_realtime.deepsort_tracker import DeepSort
from PIL import Image

# Load YOLO model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLO model
tracker = DeepSort(max_age=30, n_init=3, nn_budget=100)  # Initialize DeepSORT tracker

def extract_and_track_objects(frame_path):
    img = Image.open(frame_path).convert('RGB')
    results = yolo_model(img)
    detections = results.pandas().xyxy[0]
    
    # Filter crime-related objects (e.g., knife, gun, person)
    crime_objects = detections[detections['name'].isin(['knife', 'gun', 'person'])]
    
    # Perform tracking using DeepSORT
    bboxes = crime_objects[['xmin', 'ymin', 'xmax', 'ymax']].values
    confidences = crime_objects['confidence'].values
    track_ids = tracker.update_tracks(bboxes, confidences=confidences, frame=img)
    
    return track_ids

# Example usage
if __name__ == "__main__":
    frame_path = "data/frames/frame_0.jpg"
    track_ids = extract_and_track_objects(frame_path)
    print("Tracked Objects:", track_ids)


In [ ]:
import cv2
import pytesseract

crime_keywords = ["gun", "help", "shoot", "suspect", "robbery", "assault", "scream", "emergency"]

def extract_and_filter_text(frame_path):
    img = cv2.imread(frame_path)
    text = pytesseract.image_to_string(img)
    
    # Check if any crime-related words exist in the extracted text
    crime_related_text = [word for word in crime_keywords if word.lower() in text.lower()]
    return crime_related_text

# Example usage
if __name__ == "__main__":
    frame_path = "data/frames/frame_0.jpg"
    crime_text = extract_and_filter_text(frame_path)
    print("Crime-Related Text:", crime_text)


In [ ]:
import cv2
import os
from extract_frames import extract_frames_from_video
from extract_visual_features import extract_and_track_objects
from extract_textual_features import extract_and_filter_text


def combine_videos_and_generate_output(video_paths, output_video_path, frame_rate=1, resolution=(640, 480)):
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Video codec
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, resolution)  # Create video writer

    for video_path in video_paths:
        output_dir = "data/frames"  # Directory to save frames from the video
        extract_frames_from_video(video_path, output_dir, frame_rate)
        
        for frame_file in sorted(os.listdir(output_dir)):
            frame_path = os.path.join(output_dir, frame_file)
            
            # Process each frame for tracking and text extraction
            track_ids = extract_and_track_objects(frame_path)
            crime_text = extract_and_filter_text(frame_path)
            
            # Visualize the frame with tracking information
            frame = cv2.imread(frame_path)
            
            # Draw bounding boxes and track IDs on the frame
            for track in track_ids:
                x1, y1, x2, y2, track_id = track
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                cv2.putText(frame, f"ID:{int(track_id)}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            
            # Add crime-related text info
            if crime_text:
                cv2.putText(frame, ' '.join(crime_text), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            # Write the processed frame to the output video
            out.write(frame)

    out.release()  # Release the video writer

# Example usage
if __name__ == "__main__":
    video_paths = ["data/video/crime_scene_video_1.mp4", "data/video/crime_scene_video_2.mp4"]
    output_video_path = "data/output/crime_scene_analysis_output.avi"
    combine_videos_and_generate_output(video_paths, output_video_path, frame_rate=5)
    print("Output video saved as:", output_video_path)
